In [8]:
import requests
from xml.etree import ElementTree

import pandas as pd

import torch
import torch.nn as nn
import torch.optim as optim

from torchtext.datasets import UDPOS

import numpy as np
import math

import time
import random
from torchtext.vocab import build_vocab_from_iterator
from torch.utils.data import Dataset, DataLoader


In [6]:
# Initialize the dataset
pub_med = []
num_articles = 10000 # want as many as there are with the correct information

# Use ESearch to get PMIDs
esearch_url = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi"
search_params = {
    'db': 'pubmed',
    'term': 'intelligence[Abstract] AND ("2013/01/01"[Date - Publication] : "2023/12/31"[Date - Publication])',
    'retmax': num_articles, # is by default 20
    'retmode': 'json',

}

# send request to eSearch, and process response
search_response = requests.get(esearch_url, params=search_params)
search_data = search_response.json()['esearchresult']
pmids = search_response.json()['esearchresult']['idlist'] # pmid are the identifyers of the different articles


In [7]:
# how many articles with "intelligence in the abstract
total_count = search_data['count']  # Fetch the total count
print(f"Total number of articles with 'intelligence' in the abstract published between 2013 to 2023: {total_count}")
print(len(pmids))

Total number of articles with 'intelligence' in the abstract published between 2013 to 2023: 197330
9999


In [10]:
# eSearch is to find identifiers, eFetch is to find the abstracts
efetch_url = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi"

# Process in batches due to API limits
for i in range(0, len(pmids), 100):
    batch_pmids = pmids[i:i+100]
    fetch_params = {
        'db': 'pubmed',
        'id': ','.join(batch_pmids),
        'retmode': 'xml'
    }

    fetch_response = requests.get(efetch_url, params=fetch_params)

    if fetch_response.status_code == 200:
        root = ElementTree.fromstring(fetch_response.content)
        for article in root.findall(".//PubmedArticle"):
            pmid = article.find(".//PMID").text
            article_title = article.find(".//ArticleTitle").text
            abstract_text = article.find(".//Abstract/AbstractText").text if article.find(".//Abstract/AbstractText") is not None else "No abstract available"
            pub_date = article.find(".//PubDate/Year").text if article.find(".//PubDate/Year") is not None else "No publication year"
            pub_med.append({'PMID': pmid, 'Title': article_title, 'Abstract': abstract_text, 'Publication Year': pub_date})

# Added pmids to be added to the list, so it gets included in the DataFrame
# Convert to DataFrame
pub_med_df = pd.DataFrame(pub_med)

In [11]:
print(pub_med_df.head())
print(f"Total articles in the dataset: {len(pub_med_df)}")

       PMID                                              Title  \
0  38336420  The impact of austerity on children: Uncoverin...   
1  38333244  Artificial intelligence-powered intraoperative...   
2  38333196  Helix-based screening with structure predictio...   
3  38333180  Role of robotics and artificial intelligence i...   
4  38333148  Relationship between spiritual intelligence an...   

                                            Abstract Publication Year  
0  Which children are most vulnerable when their ...             2024  
1                              No abstract available             2024  
2  The rapid development of drugs against emergin...             2024  
3  Artificial intelligence or AI may be identifie...             2023  
4  The COVID-19 pandemic has caused physical and ...             2023  
Total articles in the dataset: 9997


In [12]:
pub_med_df.to_csv('pubmed_articles_first_9999.csv', index=False)

## Data processing

##Pipeline:
Promt goes into UI (streamlit?), which has a search function which interacts with ElasticSearch, which then gives relevant results, which are then displayed in the UI

In [7]:
# !pip install -U sentence-transformers

     ---------------------------------------- 0.0/86.0 kB ? eta -:--:--
     ---------------------------------------- 86.0/86.0 kB 2.4 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ---------------------------------------- 0.0/977.6 kB ? eta -:--:--
     ------------------------------ ------ 809.0/977.6 kB 12.7 MB/s eta 0:00:01
     -------------------------------------- 977.6/977.6 kB 8.9 MB/s eta 0:00:00
   ---------------------------------------- 0.0/1.1 MB ? eta -:--:--
   ------------------------ --------------- 0.7/1.1 MB 14.4 MB/s eta 0:00:01
   ---------------------------------------  1.1/1.1 MB 14.4 MB/s eta 0:00:01
   ---------------------------------------- 1.1/1.1 MB 12.0 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125953 sha256=e5943f802e360fd009241cca5bb8b6169336419422cb36785911bb137158ff9a
  Stored in directory: c:\users\

In [8]:
# ! pip install transformers

In [30]:
# ! pip install elasticsearch

In [31]:
# ! pip install opensearch

In [32]:
# !pip install python-dotenv

In [9]:
# Want to access elastic search from notebook:
from elasticsearch import Elasticsearch
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('all-MiniLM-L6-v2')



Get data from Pub_Med, process it in pandas -> Then convert into vectors (using BERT?), not convert everything, just abstract -> insert everything into elasticSearch to process queries. Make UI in streamlit?

Write a search function in streamlit UI,
So for a key word or prompt, it goes into streamlit UI, into search function which talks to elasticsearch, which gives relevant results and shows them in the UI.


In [12]:
# Since I didnt get elasticsearch to work yet, I wanted to make a simple model that would still work without it
# First make abstract embeddings, which are vector representatives of the abstracts, given by the model. 
abstracts = pub_med_df['Abstract'].tolist()
abstract_embeddings = model.encode(abstracts, convert_to_tensor=True)


KeyboardInterrupt: 

In [14]:
# to get an estimation for how long it will take to run, try with a smaller number first
num_samples = 20
current_num_abstracts = len(pub_med_df)

sample_abstracts = abstracts[:num_samples]

start_time = time.time()

sample_embeddings = model.encode(sample_abstracts, convert_to_tensor=True)

end_time = time.time()

time_taken = end_time - start_time

# here all is if we included every article from pubmed that meets the requirements, not all in our dataframe
estimated_time_for_all = time_taken * (int(total_count) / num_samples)
estimated_time_for_current_amount_abstracts = time_taken * (current_num_abstracts / num_samples)


NameError: name 'abstracts' is not defined

In [22]:

print(f'It took {time_taken:.2f} seconds to complete {num_samples:.2f} embeddings.')


print(f'The estimated time for all abstracts is then {estimated_time_for_all / (60 * 60):.2f} hours and for current length of df: {estimated_time_for_current_amount_abstracts / (60 * 60):.2f} hours')
print("This is assuming we have a good representative sample on article length")

It took 6.27 seconds to complete 20.00 embeddings.
The estimated time for all abstracts is then 17.10 hours and for current length of df: 0.87 hours
This is assuming we have a good representative sample on article length


In [33]:
# Making a rudimentary search function
def find_relevant_articles(question, top_k=5):
    # the question is also turned into a multidimensional vector, like the abstracts
    question_embedding = model.encode(question, convert_to_tensor=True)
    # we can then find how similar the question is to the different abstracts, using Cosine Similarity. [0, 1], where closer to one is more similar
    cos_scores = util.pytorch_cos_sim(question_embedding, sample_embeddings)[0] # currently using sample embeddings, so very few
    top_results = torch.topk(cos_scores, k=top_k)

    print("Question:", question)
    print("\nTop relevant articles:")
    for score, idx in zip(top_results[0], top_results[1]):
        idx = idx.item()
        print(f"Article Index: {idx}, Title: {pub_med_df.iloc[idx]['Title']}, Abstract: {pub_med_df.iloc[idx]['Abstract']}, (Score: {score:.4f})")


In [34]:

find_relevant_articles("How is covid related to intelligence?")

Question: How is covid related to intelligence?

Top relevant articles:
Article Index: 11, Title: Global research on artificial intelligence in thyroid-associated ophthalmopathy: A bibliometric analysis., Abstract: To provide an overview of global publications on artificial intelligence (AI) in thyroid-associated ophthalmopathy (TAO) through bibliometric analysis., (Score: 0.3076)
Article Index: 13, Title: Impact of Epidemic Intelligence Service Training in Occupational Respiratory Epidemiology., Abstract: The Centers for Disease Control and Prevention's Epidemic Intelligence Service (EIS) is a fellowship in applied epidemiology for physicians, veterinarians, nurses, scientists, and other health professionals. Each EIS fellow is assigned to a position at a federal, state, or local site for 2 years of on-the-job training in outbreak investigation, epidemiologic research, surveillance system evaluation, and scientific communication. Although the original focus of the program on the contr

__Comment:__ As we can see the Cosine Similarity is low, the largest is 0.3, but this rudimentary version actually works. The following objective is to expand the model and the search, implementing better search functions, and hopefully get an elastic search server to work. 